In [8]:
import tensorflow as tf
import h5py
import numpy as np
import face1






filename = 'tempFiles/embedded1.hdf5'
f = h5py.File(filename, 'r')
dic = {}
for raw in f:
    for j in f[raw]:
        dic[str(raw),int(j)] = f[raw][j][:] 
        
f.close()
metadata = face1.metadata
num = len(metadata)
embedded = np.empty(num)
embedded2 = np.empty(num)



embedded = np.load("tempFiles/embedding1.npy")
embedded2 = np.load("tempFiles/embedding2.npy")


filename = 'tempFiles/embedded2.hdf5'
f = h5py.File(filename, 'r')
mic = {}
for raw in f:
        for j in f[raw]:
            xc = j.split('.')
            mic[str(raw),int(xc[0])] = f[raw][j][:]
        
f.close()
    


from sklearn.metrics import f1_score, accuracy_score

distances = [] # squared L2 distance between pairs
identical = [] # 1 if same identity, 0 otherwise
metadata = face1.metadata
num = len(metadata)



from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import svm



train_in =np.ones((metadata.shape[0]), dtype=bool)
test_in =np.ones((metadata.shape[0]), dtype=bool)
n1 =None
n2 =None
name=None
for i,m in enumerate(metadata):    # 90 / 10 dataset 
        if name==None:
            name=m.name
            train_in[i]=False
            test_in[i]=True
            continue
        if name==m.name:
            train_in[i]=True
            test_in[i]=False
        else:
            name=m.name
            train_in[i]=False
            test_in[i]=True



targets2 = np.array([m.name for m in metadata])

encoder1 = LabelEncoder()
encoder1.fit(targets2)

y = encoder1.transform(targets2)


# 50 train examples of 10 identities (5 examples each)
X_train = embedded2[train_in]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded2[test_in]

y_train = y[train_in]
y_test = y[test_in]





In [9]:
def neural_net(X_train,X_test,y_train,y_test):
    sess = tf.Session()
    learning_rate = 0.55
    num_steps = 500
    batch_size = 128
    display_step = 20

    # Network Parameters
    n_hidden_1 =120# 1st layer number of neurons
    n_hidden_2 = 120# 2nd layer number of neurons
    num_input = 128 # MNIST data input (img shape: 28*28)
    num_classes = 9 # MNIST total classes (0-9 digits)

    # tf Graph input
    X = tf.placeholder("float", [None, num_input])
    Y = tf.placeholder("float", [None, num_classes])

    weights = {
        'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_classes]))
    }

    def neural_net(x):
        # Hidden fully connected layer with 256 neurons
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        # Output fully connected layer with a neuron for each class
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

    logits = neural_net(X)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


    x_data = X_train.astype(np.float32)
    y_data = y_train.astype(np.float32)
    y_data=y_data.reshape(y_data.shape[0],1)
    y_data=tf.one_hot(y_data, 9,
               on_value=1.0, off_value=0.0,
               axis=-1)
    y_data=sess.run(tf.reshape(y_data,[y_data.shape[0],9]))


    x_t=X_test.astype(np.float32)
    y_t = y_test.astype(np.float32)

    y_t= y_t.reshape(y_t.shape[0],1)
    y_t=tf.one_hot(y_t, 9,
               on_value=1.0, off_value=0.0,
               axis=-1)

    y_t=sess.run(tf.reshape(y_t,[y_t.shape[0],9]))

    init = tf.global_variables_initializer()
    sess.run(init)


    for step in range(1, num_steps+1):
        batch_x, batch_y = x_data,y_data
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
    #         print("Step " + str(step) + ", Minibatch Loss= " + \
    #               "{:.4f}".format(loss) + ", Training Accuracy= " + \
    #               "{:.3f}".format(acc))

    # print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Neural_network Accurancy:", \
        sess.run(accuracy, feed_dict={X:x_t,
                                      Y: y_t}))





In [10]:
def random_forest(X_train,X_test,y_train,y_test):
    x_data = X_train.astype(np.float32)
    y_data = y_train.astype(np.float32)



    x_t=X_test.astype(np.float32)
    y_t = y_test.astype(np.float32)
    # Parameters
    from tensorflow.contrib.tensor_forest.python import tensor_forest
    from tensorflow.python.ops import resources
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

    num_steps = 500 # Total steps to train
    batch_size = 1024 # The number of samples per batch
    num_classes = 9 # The 10 digits
    num_features = 128 # Each image is 28x28 pixels
    num_trees = 70
    max_nodes = 2000
    tf.reset_default_graph() 
    # Input and Target data
    X = tf.placeholder(tf.float32, shape=[None, num_features])
    # For random forest, labels must be integers (the class id)
    Y = tf.placeholder(tf.int32, shape=[None])

    # Random Forest Parameters
    hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                          num_features=num_features,
                                          num_trees=num_trees,
                                          max_nodes=max_nodes).fill()

    # Build the Random Forest
    forest_graph = tensor_forest.RandomForestGraphs(hparams)
    # Get training graph and loss
    train_op = forest_graph.training_graph(X, Y)
    loss_op = forest_graph.training_loss(X, Y)

    # Measure the accuracy
    infer_op, _, _ = forest_graph.inference_graph(X)
    correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Initialize the variables (i.e. assign their default value) and forest resources
    init_vars = tf.group(tf.global_variables_initializer(),
        resources.initialize_resources(resources.shared_resources()))

    # Start TensorFlow session
    sess = tf.Session()

    # Run the initializer
    sess.run(init_vars)

    # Training
    for i in range(1, num_steps + 1):
        # Prepare Data

        _, l = sess.run([train_op, loss_op], feed_dict={X: x_data, Y: y_data})
#         if i % 50 == 0 or i == 1:
#             acc = sess.run(accuracy_op, feed_dict={X: x_data, Y: y_data})
#             print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))


    print("Random_forest Accuracy:", sess.run(accuracy_op, feed_dict={X: x_t, Y: y_t}))

In [11]:
def classifier(X_train,X_test,y_train,y_test):
    knn1 = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

    svc1 = svm.SVC(kernel='rbf', gamma=1, C=1.0, probability=True)

    knn1.fit(X_train, y_train)
    svc1.fit(X_train, y_train)

    acc_knn1 = accuracy_score(y_test, knn1.predict(X_test))
    acc_svc1 = accuracy_score(y_test, svc1.predict(X_test))
    print(f'KNN accuracy = {acc_knn1}, SVM accuracy = {acc_svc1}')

In [12]:
random_forest(X_train,X_test,y_train,y_test)
neural_net(X_train,X_test,y_train,y_test)
classifier(X_train,X_test,y_train,y_test)

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 70, 'max_nodes': 2000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 11, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_classes': 9, 'num_features': 128, 'bagged_num_features': 128, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 10, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}
Random_forest Accuracy: 1.0
Neur